# ILLUSTRATION OF STELLAR EVOLUTION

The following tool gives you the oportunity to investigate how changes in the global stellar parameters affect other properties of the star. You can change the stellar mass (_M_), age, and metallicty (\[Fe/H\]):

\begin{equation}
\mathrm{[Fe/H]} = \log_{10}\left( \frac{X_\mathrm{S}}{Z_\mathrm{S}} \right) - \log_{10}\left( \frac{X_\mathrm{S}}{Z_\mathrm{S}} \right)_\odot
\end{equation}


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['text.usetex'] = True
import ipywidgets as widgets
import matplotlib.animation as ani
from matplotlib import rc
from IPython.display import HTML, display
import glob
#import textwrap as tw
rcParams['animation.html'] = 'html5'

def read_history(name):

    # Function to read history files from MESA
    
    dct = {}
    f = open(name)
    for i, line in enumerate(f):
        if i == 5:
            keys = line.split()
            break
    f.close()
    data = np.genfromtxt(name,skip_header=5)
    data= data[~np.isnan(data).any(axis=1)]
    
    for j, key in enumerate(keys):
        dct[key] = data[:,j]

    return dct

In [2]:
# Function to read gyre output

def gyre_read(name):
    data = np.genfromtxt(name,skip_header=5)
    # See https://bitbucket.org/rhdtownsend/gyre/wiki/Output%20Files%20(5.0)
    l = data[:,0]
    n = data[:,2]
    v = data[:,4]
    I = data[:,7]

    mask0 = l == 0
    mask1 = l == 1
    mask2 = l == 2

    return l[mask0], n[mask0], v[mask0], I[mask0], l[mask1], n[mask1], v[mask1], I[mask1], l[mask2], n[mask2], v[mask2], I[mask2]

def echelle(name):
    l0, n0, v0, I0, l1, n1, v1, I1, l2, n2, v2, I2 = gyre_read(name)
    mdnu = np.mean(np.diff(v0))
    x0 = np.mod(v0,mdnu)
    x1 = np.mod(v1,mdnu)
    x2 = np.mod(v2,mdnu)
    return mdnu, x0, v0, x1, v1, x2, v2

In [3]:
# Read details about the grid from summary file

Mass = []
FeH = []
where = []
with open("./OUTREACH_GRID/overview.txt") as f:
    lines = f.readlines()
    for line in lines:
        words = line.split()
        Mass.extend([float(words[0])])
        FeH.extend([float(words[1])])
        where.extend([words[2]])

In [4]:
# Introduce sliders, so that use can choose both mass and metallicity of the star

setmass = widgets.FloatSlider(
    value=max(Mass)/2,
    min=min(Mass),
    max=max(Mass),
    step=0.1,
    description=r'$M/\mathrm{M}_\odot:$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
display(setmass)

setFeH = widgets.FloatSlider(
    value=max(FeH)/2,
    min=min(FeH),
    max=max(FeH),
    step=0.1,
    description=r'$\mathrm{[Fe/H]}:$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
display(setFeH)

FloatSlider(value=1.5, description='$M/\\mathrm{M}_\\odot:$', max=3.0, min=0.6, readout_format='.1f')

FloatSlider(value=0.3757438507564905, description='$\\mathrm{[Fe/H]}:$', max=0.751487701512981, min=-2.8325486…

Here, you can change the stellar mass and metallicity. The script will then look for the closest match in our grid.

In [5]:
# Find the closest existing model in the grid

ii = ((np.asarray(Mass)-setmass.value)**2+(np.asarray(FeH)-setFeH.value)**2).argmin()

In [6]:
# We will allow the user either to choose a specific age of the star or to generate a movie of the evolution

evol = widgets.Checkbox(
    value=True,
    description='Turn on evolution:',
    disabled=False
)
display(evol)

Checkbox(value=True, description='Turn on evolution:')

Here, you can choose whether or not to "turn on the evolution". If you decide to do so, the script will generate a video, showing how the stellar properties evolve with time (this might take half a minute). Otherwise, you will be asked to choose an age on slider, which will generate a single plot.

In [7]:
# Read in the properties of the model from the history file

dct = read_history(where[ii]+"LOGS/history.data")
nr = dct["model_number"]
age = dct["star_age"]/1e6
M = dct["star_mass"]
R = dct["photosphere_r"]
L = dct["luminosity"]
Teff = dct["effective_T"]
numax = 3090*M/R**2/np.sqrt(Teff)
G  = 6.6743e-8
Mo = 1.98847e33
Ro = 6.96340e10
logg = np.log10(G*(M*Mo)/(R*Ro)**2)
dnu_scal = 135.1*(M*Mo)**(1./2.)*(R*Ro)**(-3./2.)

In [8]:
# Associate numbers with profile numbers

Profiles = glob.glob(where[ii]+"LOGS/*.index")
Profiles = np.genfromtxt(Profiles[0],skip_header=1)
nrpr = Profiles[:,0]
prof = Profiles[:,2]

# Which profiles have frequencies (all that are all in the history file)

ef = [j for j, n in enumerate(nrpr) if n in nr]

prof = prof[ef]

In [9]:
# If a movie is not requested, we generate a slider to choose the age

if not evol.value:
    setage = widgets.FloatSlider(
        value=max(age)/2,
        min=min(age),
        max=max(age),
        step=0.1,
        description=r'$\mathrm{Age \,\, [Myr]:}$',
        disabled=False,
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
    )
    display(setage)

In [10]:
# The evolution track is plotted

fig, ax = plt.subplots(2,figsize=(10,12))
ax[0].semilogy(Teff,L,lw=2)
ax[0].set_xlim(max(Teff)+50,min(Teff)-50)
ax[0].set_xlabel(r'$T_\mathrm{eff} \,\, \mathrm{[K]}$',fontsize=16)
ax[0].set_ylabel(r'$L \,\, [\mathrm{L_\odot}]$',fontsize=16)
ax[0].tick_params(axis='both', which='major', labelsize=16)
ax[0].tick_params(axis='both', which='minor', labelsize=16)

ax[1].set_xlabel(r"$\nu_{n\ell} \,\,\mathrm{mod}\,\,\Delta \nu\,\,\mathrm{[\mu Hz]}$",fontsize=16)
ax[1].set_ylabel(r"$\nu_{n\ell}\,\,\mathrm{[\mu Hz]}$",fontsize=16)
ax[1].tick_params(axis='both', which='major', labelsize=16)
ax[1].tick_params(axis='both', which='minor', labelsize=16)

plt.gcf().subplots_adjust(bottom=0.18)
plt.gcf().subplots_adjust(left=0.18)
plt.gcf().subplots_adjust(right=0.75)

tsize  = 16

# Add text to plot

x = 1.1
y = 2

M_text = plt.text(x, y, r'$M/\mathrm{M}_\odot= \,$'+str(round(M[0],1)), 
                 fontsize=tsize,transform=ax[1].transAxes)
F_text = plt.text(x, y-0.2, r'$\mathrm{Fe/H}= \,$'+str(round(FeH[ii],1)), 
                 fontsize=tsize,transform=ax[1].transAxes)
numax_text = plt.text(x, y-0.4, r'$\nu_\mathrm{max}= \,$'+str(round(numax[0],1))+r"$\,\mu \mathrm{Hz}$", 
                     fontsize=tsize,transform=ax[1].transAxes)
dnu_text = plt.text(x, y-0.6, r'$\Delta \nu \,=$'+str(round(dnu_scal[0],1))+r"$\,\mu \mathrm{Hz}$", 
                   fontsize=tsize,transform=ax[1].transAxes)
logg_text = plt.text(x, y-0.8, r'$\log g = \,$'+str(round(logg[0],1))+r"$\,\mathrm{dex}$", 
                    fontsize=tsize,transform=ax[1].transAxes)
R_text = plt.text(x, y-1.0, r'$R/\mathrm{R}_\odot= \,$'+str(round(R[0],1)), 
                 fontsize=tsize,transform=ax[1].transAxes)
age_text = plt.text(x, y-1.2, r'$\mathrm{Age} = \,$'+str(round(age[0],1))+r"$\,\mathrm{Myr}$", 
                   fontsize=tsize,transform=ax[1].transAxes)

# Is an evolution movie requested or not?
if not evol.value:
    iage = abs(age-setage.value).argmin()
    ax[0].semilogy(dct["effective_T"][iage],dct["luminosity"][iage],"ro")

#    gyre_file = where[ii] + "FREQS/profile" + str(int(prof[iage])) + ".data.GYRE.sgyre_l"
    gyre_file = where[ii] + "FREQS/" + where[ii][len("./OUTREACH_GRID/"):-1] + "_n" + str(int(prof[iage])) + ".profile.FGONG.sgyre_l0"

    mdnu, x0, v0, x1, v1, x2, v2 = echelle(gyre_file)
    
    ax[1].plot(x0, v0,"bo",label=r"$\ell = 0$")
#    if list(v1):
#        ax[1].plot(x1, v1,"r>",label=r"$\ell = 1$")
#    if list(v2):
#        ax[1].plot(x2, v2,"gs",label=r"$\ell = 2$")
    
    ax[1].set_xlim(0,mdnu)
    
    # Reset text
    numax_text.set_text(r'$\nu_\mathrm{max}= \,$'+str(round(numax[iage],1))+r"$\,\mu \mathrm{Hz}$")
    dnu_text.set_text(r'$\Delta \nu \,=$'+str(round(mdnu,1))+r"$\,\mu \mathrm{Hz}$")
    logg_text.set_text(r'$\log g = \,$'+str(round(logg[iage],1))+r"$\,\mathrm{dex}$")
    R_text.set_text(r'$R/\mathrm{R}_\odot= \,$'+str(round(R[iage],1)))
    age_text.set_text(r'$\mathrm{Age} = \,$'+str(round(age[iage],1))+r"$\,\mathrm{Myr}$")
else:
    ages = np.linspace(min(age),max(age),200)
    pt, = ax[0].plot([], [], "ro")
    echelle0, = ax[1].plot([],[],"bo",label=r"$\ell = 0$")
    plt.legend(loc=2,fontsize=16)
    
    def init():
        pt.set_data([], [])
        return (pt,)
    def animate(i):
        iage = abs(age-ages[i]).argmin()
        x = Teff[iage]
        y = L[iage]
        pt.set_data(x, y)

#        gyre_file = where[ii] + "FREQS/profile" + str(int(prof[iage])) + ".data.GYRE.sgyre_l"
        gyre_file = where[ii] + "FREQS/" + where[ii][len("./OUTREACH_GRID/"):-1] + "_n" + str(int(prof[iage])) + ".profile.FGONG.sgyre_l0"
        
        mdnu, x0, v0, x1, v1, x2, v2 = echelle(gyre_file)

        # Animations only work for one x and y array, which makes it somewhat 
        # cumbersome when you have two subplots. Here, a simple hack
        del ax[1].lines[0]
        ax[1].plot(x0, v0,"bo")
        ax[1].set_xlim(0,mdnu)
        ax[1].set_ylim(0.9*min(v0),1.1*max(v0))
        
        numax_text.set_text(r'$\nu_\mathrm{max}= \,$'+str(round(numax[iage],1))+r"$\,\mu \mathrm{Hz}$")
        dnu_text.set_text(r'$\Delta \nu \,=$'+str(round(mdnu,1))+r"$\,\mu \mathrm{Hz}$")
        logg_text.set_text(r'$\log g = \,$'+str(round(logg[iage],1))+r"$\,\mathrm{dex}$")
        R_text.set_text(r'$R/\mathrm{R}_\odot= \,$'+str(round(R[iage],1)))
        age_text.set_text(r'$\mathrm{Age} = \,$'+str(round(age[iage],1))+r"$\,\mathrm{Myr}$")
        return (pt,)

    anim = ani.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=100, blit=True)
    

    video = anim.to_html5_video()
    html = HTML(video)
    display(html)
    plt.close()

    anim.save('myAnimation2.gif', writer='imagemagick', fps=30)   